<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_tower_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行方法

1. 設定に絞り込みたいeNB-LCIDを入力
2. 上部メニューより「ランタイム」を選択
3. 「すべてのセルを実行」を選択
4. アップロードで一時停止しているので参照ボタンを押してTowerCollectorのCSVファイルをアップロード
5. 最後にマップが表示される

In [1]:
f = lambda enb, lcid: (enb << 8) | lcid

# 設定

In [2]:
# eNB-LCID

cells = [f(344574, 1)]

# 複数の場合
# cells = [f(737430, 10), f(737430, 11), f(737430, 12)]

cells

[88210945]

# アップロード

参照ボタンを押してCSVファイルをアップロード

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving 2022-05-05-23-40-47.csv to 2022-05-05-23-40-47.csv
User uploaded file "2022-05-05-23-40-47.csv" with length 52942 bytes


# プログラム

In [4]:
!pip install -U pandas

In [5]:
import pandas as pd

In [6]:
import folium
from folium.features import DivIcon

In [7]:
def make_map(df, zoom=14):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    map = folium.Map(tiles = None, location=[lat, lon], zoom_start=zoom)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>",
    ).add_to(map)

        # Googleマップ航空写真
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map(航空写真)",
        attr="<a href='https://developers.google.com/maps/documentation'>© Google</a>",
    ).add_to(map)

    for i, r in df.iterrows():

        folium.CircleMarker(
            location=[r.lat, r.lon],
            radius=10,
            color=r.color,
            fill=True,
            tooltip=r["id"],
        ).add_to(map)

        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="text-align:center; font-size: 12pt; font-weight: bold">{r["ta"]}</div>',
            ),
            tooltip=f'{r["id"]}<br />TA:{r["ta"]}<br />RSRP:{r["rsrp"]}',
        ).add_to(map)
    
    folium.LayerControl().add_to(map)

    return map

# データラングリング

In [8]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"], dtype={"ta": "Int64"}).dropna(how="all", axis=1)
df0

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,measured_at,net_type,neighboring,device,rsrp,rsrq,rssi,rssnr,ec_no,arfcn
0,440,11,13314,71694342,280056,6,260,17,-79,5,...,2022-05-05 06:57:51.906000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-79.0,14.0,NaN,1500.0
1,440,11,13314,71694342,280056,6,260,15,-83,5,...,2022-05-05 06:58:27.885000+00:00,LTE,False,Xiaomi XIG02,-113.0,-11.0,-83.0,10.0,NaN,1500.0
2,440,11,13314,71694342,280056,6,260,18,-77,6,...,2022-05-05 06:59:03.863000+00:00,LTE,False,Xiaomi XIG02,-110.0,-12.0,-77.0,9.0,NaN,1500.0
3,440,11,13314,71694342,280056,6,260,16,-81,5,...,2022-05-05 07:01:14.854000+00:00,LTE,False,Xiaomi XIG02,-115.0,-13.0,-81.0,6.0,NaN,1500.0
4,440,11,13314,71694342,280056,6,260,18,-77,5,...,2022-05-05 07:01:32.863000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-77.0,6.0,NaN,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:31.843000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
292,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:35.845000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
293,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:39.847000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
294,440,11,13315,71660291,279923,3,337,20,-73,6,...,2022-05-05 07:56:44.843000+00:00,LTE,False,Xiaomi XIG02,-106.0,-11.0,-73.0,6.0,NaN,1500.0


In [9]:
df0.dtypes

mcc                            int64
mnc                            int64
lac                            int64
cell_id                        int64
short_cell_id                  int64
rnc                            int64
psc                            int64
asu                            int64
dbm                            int64
ta                             Int64
lat                          float64
lon                          float64
accuracy                     float64
speed                        float64
bearing                      float64
altitude                     float64
measured_at      datetime64[ns, UTC]
net_type                      object
neighboring                     bool
device                        object
rsrp                         float64
rsrq                         float64
rssi                         float64
rssnr                        float64
ec_no                        float64
arfcn                        float64
dtype: object

In [10]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,measured_at,net_type,neighboring,device,rsrp,rsrq,rssi,rssnr,ec_no,arfcn
0,440,11,13314,71694342,280056,6,260,17,-79,5,...,2022-05-05 06:57:51.906000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-79.0,14.0,NaN,1500.0
1,440,11,13314,71694342,280056,6,260,15,-83,5,...,2022-05-05 06:58:27.885000+00:00,LTE,False,Xiaomi XIG02,-113.0,-11.0,-83.0,10.0,NaN,1500.0
2,440,11,13314,71694342,280056,6,260,18,-77,6,...,2022-05-05 06:59:03.863000+00:00,LTE,False,Xiaomi XIG02,-110.0,-12.0,-77.0,9.0,NaN,1500.0
3,440,11,13314,71694342,280056,6,260,16,-81,5,...,2022-05-05 07:01:14.854000+00:00,LTE,False,Xiaomi XIG02,-115.0,-13.0,-81.0,6.0,NaN,1500.0
4,440,11,13314,71694342,280056,6,260,18,-77,5,...,2022-05-05 07:01:32.863000+00:00,LTE,False,Xiaomi XIG02,-109.0,-11.0,-77.0,6.0,NaN,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:31.843000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
292,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:35.845000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
293,440,11,13315,71660291,279923,3,337,21,-71,5,...,2022-05-05 07:56:39.847000+00:00,LTE,False,Xiaomi XIG02,-103.0,-11.0,-71.0,6.0,NaN,1500.0
294,440,11,13315,71660291,279923,3,337,20,-73,6,...,2022-05-05 07:56:44.843000+00:00,LTE,False,Xiaomi XIG02,-106.0,-11.0,-73.0,6.0,NaN,1500.0


In [11]:
# 日時変換
df1["measured_at"] = df1["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [12]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

In [13]:
# 色分け（青 > 緑 > 黄 > 橙 > 赤）

df1["color"] = pd.cut(
    df1["rsrp"],
    [-140, -100, -90, -80, -40],
    labels=["red", "orange", "yellow", "green"],
).astype(str)

idx = df1.groupby("cell_id")["rsrp"].transform(max) == df1["rsrp"]
df1.loc[idx, "color"] = "blue"

# 地図

## 全部

In [14]:
map1 = make_map(df1)
map1

In [15]:
map1.save("map1.html")

## 絞り込み

In [16]:
df2 = df1[df1["cell_id"].isin(cells)].copy()
df2

,mcc,mnc,lac,cell_id,short_cell_id,rnc,psc,asu,dbm,ta,...,neighboring,device,rsrp,rsrq,rssi,rssnr,ec_no,arfcn,id,color
83,440,11,11265,88210945,344574,1,0,14,-85,41,...,False,Xiaomi XIG02,-103.0,-4.0,-85.0,7.0,NaN,1575.0,344574-1,red
84,440,11,11265,88210945,344574,1,0,14,-85,41,...,False,Xiaomi XIG02,-103.0,-4.0,-85.0,7.0,NaN,1575.0,344574-1,red
85,440,11,11265,88210945,344574,1,0,14,-85,41,...,False,Xiaomi XIG02,-103.0,-4.0,-85.0,7.0,NaN,1575.0,344574-1,red
86,440,11,11265,88210945,344574,1,0,15,-83,41,...,False,Xiaomi XIG02,-100.0,-5.0,-83.0,7.0,NaN,1575.0,344574-1,red
87,440,11,11265,88210945,344574,1,0,15,-83,41,...,False,Xiaomi XIG02,-100.0,-5.0,-83.0,7.0,NaN,1575.0,344574-1,red
88,440,11,11265,88210945,344574,1,0,15,-83,41,...,False,Xiaomi XIG02,-100.0,-5.0,-83.0,7.0,NaN,1575.0,344574-1,red
89,440,11,11265,88210945,344574,1,0,11,-91,42,...,False,Xiaomi XIG02,-116.0,-11.0,-91.0,7.0,NaN,1575.0,344574-1,red
90,440,11,11265,88210945,344574,1,0,11,-91,42,...,False,Xiaomi XIG02,-116.0,-11.0,-91.0,7.0,NaN,1575.0,344574-1,red
91,440,11,11265,88210945,344574,1,0,12,-89,42,...,False,Xiaomi XIG02,-110.0,-7.0,-89.0,3.0,NaN,1575.0,344574-1,red
92,440,11,11265,88210945,344574,1,0,12,-89,42,...,False,Xiaomi XIG02,-110.0,-7.0,-89.0,3.0,NaN,1575.0,344574-1,red


In [17]:
map2 = make_map(df2, 16)
map2

In [18]:
map2.save("map2.html")

# ダウンロード

In [20]:
from google.colab import files

files.download("map2.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>